In [1]:
import os
from scipy import stats
from process import *
from utils import *

2023-11-27 10:46:43.550818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 10:46:45.745887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hongn/miniconda3/envs/ee660/lib/python3.7/site-packages/cv2/../../lib64:
2023-11-27 10:46:45.746039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hongn/miniconda3/envs/ee660/lib/python3.7/sit

## Pre-defined

In [2]:
TRIALS = {
    "all": TRIAL_LIST,
    "texture": TRIAL_TEXTURE,
    "complexity": TRIAL_COMPLEXITY,
    "orientation": TRIAL_ORIENTATION,
    "brightness": TRIAL_BRIGHTNESS,
    "contrast": TRIAL_CONTRAST,
    "color": TRIAL_COLOR,
    "depth": TRIAL_DEPTH,
    "face": TRIAL_FACE,
    "movement": TRIAL_MOVEMENT,
    "face_texture": TRIAL_FACE_TEXTURE,
}

these_trials = TRIALS["all"]
smap = "face"

def statistic_mw(x, y):
    return stats.mannwhitneyu(x, y)[0]


def iqr(x):
    q1, q3 = np.percentile(x, [25, 75])
    return q3 - q1


def extract_features(ids, trial, smap):
    if smap is not None:
        smap = trial.load_saliency_map(smap)
        smap_vs = trial.load_saliency_map("red")

    features = []
    for subject in ids:
        sub = Subject(DATA_ROOT, subject)
        out = sub.extract_fixations(trial_name=this_trial.trial_name)
        analyzer = FixationAnalyzer(DATA_ROOT, out)
        feat = analyzer.average_saliency(smap)
        feat_vs = analyzer.average_saliency(smap_vs)
        # feat = analyzer.number_of_fixations()
        # feat = sub.eye_mov_entropy(trial.trial_name, perplexity=True)
        features.append(feat - feat_vs)
        feats[subject].append(feat - feat_vs)
    return features

In [3]:
class ImageTrial:
    def __init__(self, root, trial_name, smap_dir):
        self.root = root
        self.new_res = True if "new_res" in root else False
        self.path = glob.glob(f"/mnt/c/Users/Hong/Dropbox/cvi-stimuli/*{trial_name}*")[0]
        self.trial_name = trial_name
        self.smap_dir = smap_dir
        self.ids = glob.glob(os.path.join(root, "*.asc"))
        self.ids = [os.path.basename(d)[:-4] for d in self.ids]

    def load_trial_img(self):
        img = cv2.imread(self.path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def load_saliency_map(self, smap_type):
        image_name = self.trial_name.strip(".jpg")
        filename = f"{image_name}_{smap_type}.jpg"
        path = os.path.join(self.smap_dir, image_name, filename)

        if os.path.exists(path):
            smap = plt.imread(path)
        else:
            os.makedirs(os.path.join(self.smap_dir, image_name), exist_ok=True)
            sal = SaliencyMap(smap_type)
            smap = sal.get_smap(self.load_trial_img())
            cv2.imwrite(path, smap)

        if self.new_res:
            smap = np.pad(smap, ((240, 240), (320, 320)), 'constant')
        return smap.T

    def complexity(self):
        img = self.load_trial_img()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        stats = measure.regionprops(img)
        areas = [l.area for l in stats]
        rp_tot = img.shape[0] * img.shape[1]
        return sum(areas > (rp_tot / 25000))

    def read_subjects(self, names, vel=False):
        data, frac = {}, {}
        for subject in names:
            sub = Subject(subject)
            trial_data, frac = sub.extract_data(self.trial_name, vel)
            data[subject] = trial_data
            frac[subject] = 1 - frac
        return data, frac

    def read_fixations(self, names):
        fixations = {}
        for subject in names:
            sub = Subject(subject)
            this = sub.extract_fixations(self.trial_name)
            fixations[subject] = this
        return fixations

    def extract_traces(self, names, smap):
        traces = {}
        for subject in names:
            sub = Subject(subject)
            this = sub.extract_trace(self.trial_name, smap)
            traces[subject] = this
        return traces

## Testing

In [4]:
DATA_ROOT = "/mnt/c/Users/Hong/Dropbox/cvi-asc/"

In [5]:
ids = [i for i in os.listdir(DATA_ROOT) if i.endswith(".asc")]
ctrl_ids = [i.split(".")[0] for i in ids if i.startswith("2")]
cvi_ids = [i.split(".")[0] for i in ids if i.startswith("1")]
print(f'number of control {len(ctrl_ids)} and cvi {len(cvi_ids)}')

number of control 10 and cvi 31


In [6]:
feats = {name.split(".")[0]: [] for name in ids}
trial = these_trials[2]
trial

'Freeviewingstillimage_4'

In [7]:
this_trial = ImageTrial(DATA_ROOT, trial, "smaps")
# features_ctrl = extract_features(ctrl_ids, this_trial, smap)
# features_cvi = extract_features(cvi_ids, this_trial, smap)

In [8]:
subject = cvi_ids[1]
sub = Subject(DATA_ROOT, subject)
out = sub.extract_fixations(trial_name='Freeviewingstillimage_2')
out_saccade = sub.extract_saccades(trial_name=this_trial.trial_name)
analyzer = FixationAnalyzer(DATA_ROOT, out)

In [9]:
cvi_ids

['1003_2',
 '1003_3',
 '1003_4',
 '1004_2',
 '1005_1',
 '1006_1',
 '1006_2',
 '1006_3',
 '1006_4',
 '1007_1',
 '1007_2',
 '1007_3',
 '1007_4',
 '1008_1',
 '1010_1',
 '1010_2',
 '1011_1',
 '1011_2',
 '1013_1',
 '1013_2',
 '1015_1',
 '1015_2',
 '1017_1',
 '1017_2',
 '1018_1',
 '1018_2',
 '1019_1',
 '1020_1',
 '1020_2',
 '1022_1',
 '1022_2']

In [10]:
cvi_train = {}
ctrl_train = {}
for sbjname in cvi_ids:
# subject = cvi_ids[1]
    sub = Subject(DATA_ROOT, sbjname)
    trainctmc = []
    for name in these_trials:  
        temp = sub.get_CTMC(trial_name=name)
        if(len(temp) > 2):
            trainctmc.append(temp)
    cvi_train[sbjname] = trainctmc

for sbjname in ctrl_ids:
# subject = cvi_ids[1]
    sub = Subject(DATA_ROOT, sbjname)
    trainctmc = []
    for name in these_trials:  
        temp = sub.get_CTMC(trial_name=name)
        if(len(temp) > 2):
            trainctmc.append(temp)
    ctrl_train[sbjname] = trainctmc


## CTMC

In [11]:
import jajapy as ja
from jajapy.base.Set import Set

In [12]:
# labelling = ['EBLINK','EBLINK','ESACC','EFIX','EFIX']
# transitions = [(0,1,0.08),(0,2,0.12),(1,1,0.3),(1,2,0.7),
#                 (2,0,0.2),(2,3,0.1),(2,4,0.2),(3,3,0.8),
#                 (3,1,0.1),(3,4,0.1),(4,2,0.25)]
# original_model = ja.createCTMC(transitions,labelling,initial_state=0,name="My_CTMC")

In [13]:
from jajapy.base.Set import Set
# Set(trainctmc[2], [1] * 39)

In [14]:
ctrl_train.keys()

dict_keys(['2002_1', '2002_2', '2003_1', '2003_2', '2004_1', '2004_2', '2005_1', '2005_2', '2006_1', '2006_2'])

In [15]:
trainctmc = ctrl_train['2004_2']
len(trainctmc)

38

In [16]:
for key in ctrl_train:
    print(len(ctrl_train[key]))

38
39
39
39
39
38
26
38
38
38


In [18]:
ctmc_ctrl_models = {}
ctmc_cvi_models = {}

In [22]:
nb_trials = 5
ctmc_ctrl_models = {}
ctmc_cvi_models = {}

for key in cvi_train:
    best_model = None
    quality_best = -1024
    for n in range(1,nb_trials+1):
        current_model = ja.CTMC_random(nb_states=3,
                                labelling=['EFIX','ESACC','EBLINK'],
                                self_loop=False,
                                random_initial_state=True,
                                min_exit_rate_time=1,
                                max_exit_rate_time=6)
        current_quality = current_model.logLikelihood(Set(cvi_train[key], [1] * len(cvi_train[key]), t=4))
        if quality_best < current_quality: #we keep the best model only
                        quality_best = current_quality
                        best_model = current_model
    ctmc_cvi_models[key] = ja.BW().fit(Set(cvi_train[key], [1] * len(cvi_train[key]), t=4),initial_model=best_model)

In [20]:
import pickle 

with open('ctmc_ctrl_models.pkl', 'wb') as f:
    pickle.dump(ctmc_ctrl_models, f)

In [22]:
output_model = ja.BW().fit(Set(trainctmc, [1] * len(trainctmc), t=4),initial_model=best_model)

Learning a CTMC...
|████████████████████████████████████████| 4 in 0.6s (6.98/s) 


In [21]:
output_model.matrix

NameError: name 'output_model' is not defined

In [23]:
print(output_model)

Name: CTMC_random_3_states
Initial state: s3
----STATE 0--EFIX----
Exepected waiting time: 42.54
s0 -> s1 : lambda = 0.021391631405735778091
s0 -> s2 : lambda = 0.0021156558533145275035

----STATE 1--ESACC----
Exepected waiting time: 262.79069767441860464
s1 -> s0 : lambda = 0.0038053097345132743363

----STATE 2--EBLINK----
Exepected waiting time: 65.777777777777777776
s2 -> s1 : lambda = 0.015202702702702702703

----STATE 3--init----
Exepected waiting time: 385.72972972972972971
s3 -> s0 : lambda = 0.0025924887892376681615



